In [32]:
import pandas as pd
import os
import openpyxl
from IdMapping import clean_provider_ids

In [33]:
# Define the path to the Excel file
folder_path = "../BoS (Business Objects) Raw Data Reports - Deidentified"
file_name = "143 RAW Client Data Export v3_EE Workflow.xlsx"
file_path = os.path.join(folder_path, file_name)

In [34]:
income_ent = pd.read_excel(file_path, sheet_name='INCOME ENT')
income_ext = pd.read_excel(file_path, sheet_name='INCOME EXT')
# Clean EE Provider ID
income_ent = clean_provider_ids(income_ent)
income_ext = clean_provider_ids(income_ext)
income_ent.head()

,EE Provider ID,Receiving Income (Entry),Source of Income (Entry),Monthly Income (Start),Income Start Date (Entry),Income End Date (Entry),Provider (140-provider),Date Added (140-date_added),Provider Abbrev,Provider Tree
0,"Community Housing Network, Inc. - Grafton SHU ...",Yes,SSDI (HUD),912.0,2015-12-01 12:00:00,NaT,"Community Housing Network, Inc. - Grafton SHU ...",2022-01-22 13:01:14.907,Grafton SHU,143
1,"Community Housing Network, Inc. - Grafton SHU ...",Yes,SSDI (HUD),85.0,2015-12-01 12:00:00,NaT,"Community Housing Network, Inc. - Grafton SHU ...",2022-01-22 13:06:17.519,Grafton SHU,143
2,"Community Housing Network, Inc. - Grafton SHU ...",Yes,SSDI (HUD),85.0,2015-12-01 12:00:00,NaT,"Community Housing Network, Inc. - Grafton SHU ...",2022-01-22 13:08:01.373,Grafton SHU,143
3,"Community Housing Network, Inc. - Grafton SHU ...",Yes,SSDI (HUD),85.0,2015-12-01 12:00:00,NaT,"Community Housing Network, Inc. - Grafton SHU ...",2022-01-22 13:09:25.614,Grafton SHU,143
4,"Community Housing Network, Inc. - Grafton SHU ...",No,Alimony or Other Spousal Support (HUD),NaN,2015-12-01 12:00:00,NaT,XXXCLOSED2017Lighthouse of Oakland County - Oa...,2016-02-02 19:29:55.917,Grafton SHU,143


In [35]:
income_ent.size

117800

In [36]:
income_ext.size

125520

In [37]:
start_date = pd.to_datetime('10/11/2023', format='%m/%d/%Y')
end_date = pd.to_datetime('10/10/2024', format='%m/%d/%Y')

In [38]:
# converting the date columns to datetime format and filtering the data based on the date range
income_ent['Income Start Date (Entry)'] = pd.to_datetime(income_ent['Income Start Date (Entry)'], format='%m/%d/%Y', errors='coerce')
income_ent = income_ent[(income_ent['Income Start Date (Entry)'] >= start_date) & (income_ent['Income Start Date (Entry)'] <= end_date)]

income_ext['Income Start Date (Exit)'] = pd.to_datetime(income_ext['Income Start Date (Exit)'], format='%m/%d/%Y', errors='coerce')
income_ext = income_ext[(income_ext['Income Start Date (Exit)'] >= start_date) & (income_ext['Income Start Date (Exit)'] <= end_date)]

In [39]:
income_ent.size

26730

In [40]:
income_ext.size

31120

In [41]:
# --- Group and calculate % of 'Yes' for ENTRY ---
grouped_ent = (
    income_ent
    .groupby(["Provider Abbrev", "Provider Tree"])["Receiving Income (Entry)"]
    .value_counts(normalize=True)  # Get proportions
    .rename("Percent")  # Rename column
    .reset_index()
)

# --- Group and calculate % of 'Yes' for EXIT ---
grouped_ext = (
    income_ext
    .groupby(["Provider Abbrev", "Provider Tree"])["Receiving Income (Exit)"]
    .value_counts(normalize=True)
    .rename("Percent")
    .reset_index()
)

In [42]:
# Keep only 'Yes' rows, rename for consistency, and add assessment stage
grouped_ent = grouped_ent[grouped_ent["Receiving Income (Entry)"] == "Yes"]
grouped_ent = grouped_ent.rename(columns={"Receiving Income (Entry)": "Receiving Income"})
grouped_ent["Percent"] = grouped_ent["Percent"] * 100
grouped_ent["Assessment Stage"] = "Entry"

grouped_ext = grouped_ext[grouped_ext["Receiving Income (Exit)"] == "Yes"]
grouped_ext = grouped_ext.rename(columns={"Receiving Income (Exit)": "Receiving Income"})
grouped_ext["Percent"] = grouped_ext["Percent"] * 100
grouped_ext["Assessment Stage"] = "Exit"

# Combine both groups
combined = pd.concat([grouped_ent, grouped_ext], ignore_index=True)
combined.to_excel("income_summary_by_provider.xlsx", index=False)
combined.head()

,Provider Abbrev,Provider Tree,Receiving Income,Percent,Assessment Stage
0,Grafton SHU,143,Yes,2.051282,Entry
1,Jefferson Oaks SHU,143,Yes,1.459854,Entry
2,OCHLAP1,143,Yes,12.500000,Entry
3,OLAP CG,143,Yes,6.140351,Entry
4,OLAP2,143,Yes,7.594937,Entry


In [11]:
def income_counts_entry_and_exit(folder_path, start_date, end_date):
    """
    This function filters the income at entry and exit data based on the date range provided. It creates
    a dataframe with the counts of 'Yes' and 'No' values for each program at entry and exit.
    The function also prints the value counts for each program and saves the results to an excel file.
    """
    results = [] # Store all the rows

    #Loop through all Excel files in the folder
    for file in os.listdir(folder_path):
        if file == "TEMPLATE RAW Client Data Export v3_EE Workflow.xlsx":
            continue

        if file.endswith(".xlsx"):
            file_path = os.path.join(folder_path, file)
            program_name = file.split(" ")[0]  # Extract 'Program' from filename

        try:
            income_ent = pd.read_excel(file_path, sheet_name='INCOME ENT')
            income_ext = pd.read_excel(file_path, sheet_name='INCOME EXT')
        except Exception as e:
            print(f"Error reading sheets from {file}: {e}")
            continue

        #converting the date columns to datetime format and filtering the data based on the date range
        income_ent['Income Start Date (Entry)'] = pd.to_datetime(income_ent['Income Start Date (Entry)'], format='%m/%d/%Y', errors='coerce')
        filtered_income_ent = income_ent[(income_ent['Income Start Date (Entry)'] >= start_date) & (income_ent['Income Start Date (Entry)'] <= end_date)]

        income_ext['Income Start Date (Exit)'] = pd.to_datetime(income_ext['Income Start Date (Exit)'], format='%m/%d/%Y', errors='coerce')
        filtered_income_ext = income_ext[(income_ext['Income Start Date (Exit)'] >= start_date) & (income_ext['Income Start Date (Exit)'] <= end_date)]

        yes_count_entry = filtered_income_ent['Receiving Income (Entry)'].value_counts().get('Yes', 0)
        no_count_entry = filtered_income_ent['Receiving Income (Entry)'].value_counts().get('No', 0)
        empty_count_entry = filtered_income_ent['Receiving Income (Entry)'].value_counts().get('', 0)

        yes_count_exit = filtered_income_ext['Receiving Income (Exit)'].value_counts().get('Yes', 0)
        no_count_exit = filtered_income_ext['Receiving Income (Exit)'].value_counts().get('No', 0)
        empty_count_exit = filtered_income_ext['Receiving Income (Exit)'].value_counts().get('', 0)
        #add new rows to the income_count_results dataframe
        new_entry_row_yes = ({'Receiving Income': 'Yes', 'Count': yes_count_entry, 'Assessment Stage': 'Entry', 'Program': program_name})
        new_entry_row_no = ({'Receiving Income': 'No', 'Count': no_count_entry, 'Assessment Stage': 'Entry', 'Program': program_name})
        new_entry_row_empty = ({'Receiving Income': 'Empty', 'Count': empty_count_entry, 'Assessment Stage': 'Entry', 'Program': program_name})

        new_exit_row_yes = ({'Receiving Income': 'Yes', 'Count': yes_count_exit, 'Assessment Stage': 'Exit', 'Program': program_name})
        new_exit_row_no = ({'Receiving Income': 'No', 'Count': no_count_exit, 'Assessment Stage': 'Exit', 'Program': program_name})
        new_exit_row_empty = ({'Receiving Income': 'Empty', 'Count': empty_count_exit, 'Assessment Stage': 'Exit', 'Program': program_name})

        #append the new rows to the results list
        results.append(new_entry_row_yes)
        results.append(new_entry_row_no)
        results.append(new_entry_row_empty)
        results.append(new_exit_row_yes)
        results.append(new_exit_row_no)
        results.append(new_exit_row_empty)

        # Save results to excel
        results_df = pd.DataFrame(results);
        results_df.to_excel("./Income Counts Entry and Exit.xlsx", index=False)


In [12]:
income_counts_entry_and_exit(folder_path, start_date, end_date)

In [17]:
def percent_noncash_yes(folder_path, start_date, end_date):
    """
    This function filters the non-cash data based on the date range provided. It then creates an excel file with the percent of 
    participants who report receiving non-cash benefits at entry and exit.
    """
    results = [] # Store all the rows

    #Loop through all Excel files in the folder
    for file in os.listdir(folder_path):
        if file == "TEMPLATE RAW Client Data Export v3_EE Workflow.xlsx":
            continue

        if file.endswith(".xlsx"):
            file_path = os.path.join(folder_path, file)
            program_name = file.split(" ")[0]  # Extract 'Program' from filename

        try:
            noncash_ent = pd.read_excel(file_path, sheet_name='NONCASH ENT')
            noncash_ext = pd.read_excel(file_path, sheet_name='NONCASH EXIT')

        except Exception as e:
            print(f"Error reading sheets from {file}: {e}")
            continue

        #converting the date columns to datetime format and filtering the data based on the date range
        noncash_ent['Non-Cash Start Date (Entry)'] = pd.to_datetime(noncash_ent['Non-Cash Start Date (Entry)'], format='%m/%d/%Y', errors='coerce')
        filtered_noncash_ent = noncash_ent[(noncash_ent['Non-Cash Start Date (Entry)'] >= start_date) & (noncash_ent['Non-Cash Start Date (Entry)'] <= end_date)]

        noncash_ext['Non-Cash Start Date (Exit)'] = pd.to_datetime(noncash_ext['Non-Cash Start Date (Exit)'], format='%m/%d/%Y', errors='coerce')
        filtered_noncash_ext = noncash_ext[(noncash_ext['Non-Cash Start Date (Exit)'] >= start_date) & (noncash_ext['Non-Cash Start Date (Exit)'] <= end_date)]

        #sum the number of 'Yes' values in the 'Receiving Income (Entry)' column and divide by the total number of rows in the filtered dataframe
        total_rows_entry = len(filtered_noncash_ent)
        if total_rows_entry > 0:
            num_yes = filtered_noncash_ent['Receiving Benefit (Entry)'].value_counts().get('Yes', 0)
            percent_income_at_entry = (num_yes / total_rows_entry * 100)
        else:
            percent_income_at_entry = 0

        #repeat the same process for the 'Receiving Income (Exit)' column
        total_rows_exit = len(filtered_noncash_ext)
        if total_rows_exit > 0:
            num_yes_exit = filtered_noncash_ext['Receiving Benefit (Exit)'].value_counts().get('Yes', 0)
            percent_noncash_at_exit = (num_yes_exit / total_rows_exit * 100)
        else:
            percent_noncash_at_exit = 0

        #add the program and percent_income_at_entry to the income_ent_results dataframe
        new_entry_row = ({'Program': program_name, 'Percent non-cash at entry YES': round(percent_income_at_entry, 2), 'Percent non-cash at exit YES': round(percent_noncash_at_exit, 2)})
        results.append(new_entry_row)

        # Save results to excel
        results_df = pd.DataFrame(results);
        results_df.to_excel("./Percent Non-Cash Yes at Entry and Exit.xlsx", index=False)

In [18]:
percent_noncash_yes(folder_path, start_date, end_date)

In [23]:
def calculate_median_income_everyone(folder_path, start_date, end_date):
    """This function calculates the median income at entry and exit for each program for everybody (those who report having
    income at entry and exit, and those who don't). 
    It filters the data based on the date range provided and creates an excel file with the results."""

    results = []

    #Loop through all Excel files in the folder
    for file in os.listdir(folder_path):
        if file == "TEMPLATE RAW Client Data Export v3_EE Workflow.xlsx":
            continue

        if file.endswith(".xlsx"):
            file_path = os.path.join(folder_path, file)
            program_name = file.split(" ")[0]  # Extract 'Program' from filename

        try:
            income_ent = pd.read_excel(file_path, sheet_name='INCOME ENT')
            income_ext = pd.read_excel(file_path, sheet_name='INCOME EXT')
        except Exception as e:
            print(f"Error reading sheets from {file}: {e}")
            continue

        #converting the date columns to datetime format and filtering the data based on the date range
        income_ent['Income Start Date (Entry)'] = pd.to_datetime(income_ent['Income Start Date (Entry)'], format='%m/%d/%Y', errors='coerce')
        filtered_income_ent = income_ent[(income_ent['Income Start Date (Entry)'] <= end_date) & (income_ent['Income Start Date (Entry)'] >= start_date)]

        income_ext['Income Start Date (Exit)'] = pd.to_datetime(income_ext['Income Start Date (Exit)'], format='%m/%d/%Y', errors='coerce')
        filtered_income_ext = income_ext[(income_ext['Income Start Date (Exit)'] <= end_date) & (income_ext['Income Start Date (Exit)'] >= start_date)]

        #calculate median income at entry and exit
        median_income_entry = filtered_income_ent['Monthly Income (Start)'].median()
        median_income_exit = filtered_income_ext['Monthly Income Amount (Exit)'].median()

        #add the program and median incomes to the results dataframe
        new_entry_row = ({'Program': program_name, 'Median Income at Entry': median_income_entry, 'Median Income at Exit': median_income_exit})
        results.append(new_entry_row)

        # Save results to excel
        results_df = pd.DataFrame(results);
        results_df.to_excel("./Median Income Everyone.xlsx", index=False)

****NOTE**** that median income is for everyone by program. We have not done any filtering beyond aggregating by program. We could filter by only clients who do have income--but this wasn't asked of us yet. I also didn't want to make too many changes since we might have to make more changes this week to how we filter the data.

In [24]:
calculate_median_income_everyone(folder_path, start_date, end_date)

c:\Users\alize\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


To Do:
- (COMPLETED, THOUGH LIKELY NOT ACCURATE. Create a function that calculates the median monthly income for each program a entry and exit
- (COMPLETTED) Modify the percent_income_yes() function to create a new function that calculates the percent of program participants receiving non cash benefits. We should only need to update the sheet name and confirm the column name is the same in that sheet. 

If there is time:
- Create a function that calculates the average monthly program at entry and exit
- Create a function that sums the type of employment/income sources by program. This should be similar to the income_counts_entry_and_exit() function.
- They requested a metric that showed new or increased income. This one is an optional request because they don't know if it can be calculated. 
